# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.webservice import Webservice
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset for Fraud detection in card payment is downloaded from Kaggle. It's a single CSV file with nearly 1,50,000 records. A dataset named KaggleDSFraudDS is created in workspace based on that. 

In [3]:
ws = Workspace.from_config()
experiment_name = 'CapstoneExpHyperDrive'

experiment=Experiment(ws, experiment_name)

found = False
key = "KaggleDSFraudDS"
description_text = "Kaggle DataSet for Udacity Course Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        df = dataset.to_pandas_dataframe()
        df.describe()

#Create a cluster
amlcompute_cluster_name = "cluster4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The following code tries to find the best values for hyperparameters, Inverse of regularization (C) and Max_Iter for Logistic Regression algorithm. In order to reduce the time taken while running multiple iterations, it uses BanditPolicy. BanditPolicy helps in early termination of low-performing long running jobs. The model training script is provided in Train.py file. The script reads the data file from the dataset created in the current workspace. It then removes certain columns which are not necessary for the model creation. It also converts categorical columns into dummy numeric indicators. This is required for LogisticRegression algorithm.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
etpolicy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling( {
        '--C' : choice(0.001,0.01,0.1,1.0,10.0,50.0,100,1000),
        '--max_iter': choice(50,100)
    }
)

#TODO: Create your estimator and hyperdrive config
##estimator = <your estimator here>
customenv = Environment.from_conda_specification(name='customenv', file_path='conda_dependencies.yml')
src=ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=customenv)


hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=etpolicy,
                                     max_total_runs=2,
                                     max_concurrent_runs=1)

In [5]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_42d99dac-955c-40bd-98dd-ca4128b6d721
Web View: https://ml.azure.com/runs/HD_42d99dac-955c-40bd-98dd-ca4128b6d721?wsid=/subscriptions/d18bf486-2340-49bd-9908-fbd241f929a1/resourcegroups/RGML/workspaces/wsaug&tid=06334f6d-6d96-4027-a985-f45a67d990ca

Streaming azureml-logs/hyperdrive.txt

[2022-08-12T12:58:03.394687][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-08-12T12:58:04.0473639Z][SCHEDULER][INFO]Scheduling job, id='HD_42d99dac-955c-40bd-98dd-ca4128b6d721_0' 
[2022-08-12T12:58:03.989810][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.
[2022-08-12T12:58:05.3594449Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_42d99dac-955c-40bd-98dd-ca4128b6d721_0' 
[2022-08-12T13:00:33.178077][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space
[2022-08-12T13:00:33.4611697Z][SCHEDULER][INFO]Scheduling job, id='HD_42d99dac-955c-40bd-98dd-ca4128b6d721_1' 
[2022-08-12T13:00:33.426

{'runId': 'HD_42d99dac-955c-40bd-98dd-ca4128b6d721',
 'target': 'cluster4',
 'status': 'Completed',
 'startTimeUtc': '2022-08-12T12:58:02.361811Z',
 'endTimeUtc': '2022-08-12T13:03:45.566Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3de563bb-4d8c-46da-91f0-914206d5ebf1',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1083-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.42.0',
  'space_size': '16',
  'score': '0.9582312925170068',
  'best_child_run_id': 'HD_42d99dac-955c-40bd-98dd-ca4128b6d721_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_42d99dac-955c-40bd-98dd-ca4128b6d721_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
print(best_run.get_file_names())

Run(Experiment: CapstoneExpHyperDrive,
Id: HD_42d99dac-955c-40bd-98dd-ca4128b6d721_1,
Type: azureml.scriptrun,
Status: Completed)
['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2022-08-12-13', 'outputs/model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'user_logs/std_log.txt']


In [8]:
#TODO: Save the best model
best_run.register_model( model_name='hypertraining_model',
                    model_path='outputs/model.pkl',
                    description='HyperDrive classification model')

Model(workspace=Workspace.create(name='wsaug', subscription_id='d18bf486-2340-49bd-9908-fbd241f929a1', resource_group='RGML'), name=hypertraining_model, id=hypertraining_model:2, version=2, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

The model deployment is done for the model created using AutoML. The related script is provided in the Jupyter notebook for AutoML.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

